In [1]:
import numpy as np
import pandas as pd
from typing import Literal
from scipy import stats

- The length of the one-at-a-time $t$-interval is: $t_{n-1}(\alpha/2)$
- The length of the Bonferroni $t$-interval when is: $t_{n-1}(\alpha/(2m))$
- The length of the one-at-a-time $T^{2}$-interval is: $\sqrt{\frac{(n-1)p}{n-p}F_{p, n-p}(\alpha)}$

- The length of the one-at-a-time $t$-interval when $n$ is large is approximately: $z(\alpha/2)$
- The length of the Bonferroni $t$-interval when $n$ is large is approximately: $z(\alpha/(2m))$
- The length of the one-at-a-time $T^{2}$-interval when $n$ is large is approximately: $\sqrt{\chi_{p}^{2}(\alpha)}$

In [2]:
def len_compare(alpha, num: Literal['Bon-t','t', 'T2'], denom: Literal['Bon-t','t', 'T2']):
    def bonferroni_t_len(alpha, n, m) -> float:
        # Length of the Bonferroni t-interval.
        if n == np.Inf:
            return stats.norm.ppf(1-alpha/(2*m))
        else:
            return stats.t.ppf(1-alpha/(2*m), df=n-1)
    
    def T2_len(alpha, n, p) -> float:
        # Length of the one-at-a-time T^{2} interval.
        if n == np.Inf:
            return np.sqrt(stats.chi2.ppf(1-alpha, df=p))
        else:
            return np.sqrt((((n-1)*p)/(n-p))*stats.f.ppf(1-alpha, dfn=p, dfd=n-p))
        
    def t_len(alpha, n) -> float:
        # Length of the one-at-a-time t-interval.
        if n == np.Inf:
            return stats.norm.ppf(1-alpha/2)
        else:
            return stats.t.ppf(1-alpha/2, df=n-1)
        
    n_list = [15, 25, 50, 100, np.Inf]
    p_list = [2, 4, 10]
    output_dict = dict()
    for n in n_list:
        output_dict[n] = dict()
        for p in p_list:
            m = p
            calls = {
                'Bon-t' : bonferroni_t_len(alpha, n, m),
                'T2': T2_len(alpha, n, p),
                't': t_len(alpha, n),
            }
            output_dict[n][p] = calls[num]/calls[denom]
    return pd.DataFrame(output_dict).T

In [3]:
ci_len = len_compare(alpha=0.05, num='t', denom='Bon-t')

In [4]:
ci_len

,2,4,10
15.0,0.854643,0.748880,0.644914
25.0,0.863213,0.764341,0.667817
50.0,0.869052,0.774922,0.683592
100.0,0.871799,0.779910,0.691050
inf,0.874436,0.784706,0.698233
